### modules

In [ ]:
if globals().get('LOADED_LOGGER') == None:
    %run LOGGER.ipynb

In [ ]:
if globals().get('LOADED_ANNOTATIONS') == None:
    %run ANNOTATIONS.ipynb

In [ ]:
if globals().get('LOADED_PATTERN_OBSERVER') == None:
    %run PATTERN_OBSERVER.ipynb

In [ ]:
if globals().get('LOADED_CHECKBOXES') == None:
    %run CHECKBOXES.ipynb

In [ ]:
if globals().get('LOADED_DATA_CONTAINER') == None:
    %run DATA_CONTAINER.ipynb

In [ ]:
if globals().get('LOADED_SETTINGS') == None:
    %run SETTINGS.ipynb

In [ ]:
if globals().get('LOADED_VIEWING_TABS_EXTRADAY_ANALYSIS') == None:
    %run VIEWING_TABS_EXTRADAY_ANALYSIS.ipynb

In [ ]:
if globals().get('LOADED_VIEWING_TABS_INTRADAY_ANALYSIS') == None:
    %run VIEWING_TABS_INTRADAY_ANALYSIS.ipynb

In [ ]:
if globals().get('LOADED_VIEWING_TABS_FACTOR_ANALYSIS') == None:
    %run VIEWING_TABS_FACTOR_ANALYSIS.ipynb

### import protection

In [ ]:
if globals().get('LOADED_BACKTEST_VIEWER') == None:
    logging.info('LOADED_BACKTEST_VIEWER')
    LOADED_BACKTEST_VIEWER=True

### imports

In [ ]:
from ipywidgets import HTML

### begin

In [ ]:
class BACKTEST_VIEWER(IObserver):
    
    ####################################
    # static
    ####################################
    _return_types = [
        'Cash + Future',
        'Cash',
        'Future',
    ]
    
    _view_type_dico = {
        'Extraday' : 'EXTRADAY_TAB_COLLECTION()',
        'Intraday' : 'INTRADAY_TAB_COLLECTION()',
        'Factor'   : 'FACTOR_TAB_COLLECTION()',
    }

    ####################################
    # constructor
    ####################################
    def __init__(self,
        name                                = 'BACKTEST_VIEWER',
        app_name                            = 'Backtest Viewer',
        #home_dir                            = r'C:\Users\ahkar\OneDrive\Documents\Data\Extraday\dir1',
        #home_dir                            = r'C:\Users\ahkar\OneDrive\Documents\Data\Intraday',
        home_dir                            = r'C:\Users\ahkar\OneDrive\Documents\Data\all\dir1',
        ):
        self.name                           = name
        self._app_name                      = app_name
        self._home_dir                      = home_dir
        
        # for observer pattern
        self._observers                     = []
        
        ####################################
        # instantiate SETTINGS
        ####################################
        # needs
        # - nothing
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - SETTINGS[reload_data]                                                                           [repopulate data_dico]
        # - SETTINGS[backtest_path]                           [compute backtests]
        # - SETTINGS[return_type]                                                                                                  .update plot device.
        # - SETTINGS[plot_method]                                                                                                  [update plot device]
        # - SETTINGS[view_type]                                                                             [load correct data]    [update plot device]
        # - SETTINGS[date filter]                                                                           [filter data]
        # - SETTINGS[time filter]                                                                           [filter data]
        ####################################
        self._SETTINGS = SETTINGS(
            name                            = 'SETTINGS',
            app_name                        = self._app_name,
            backtest_path                   = self._home_dir,
            view_types                      = BACKTEST_VIEWER._view_type_dico.keys(),
            return_types                    = BACKTEST_VIEWER._return_types,
            plot_methods                    = SINGLE_PLOT.plot_methods,
            width                           = '20%',
        )

        ####################################
        # instantiate BACKTEST_SELECTOR
        ####################################
        # needs
        # - SETTINGS[backtests]
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - SETTINGS[backtests]                                                      .compute books.
        ####################################
        self._BACKTEST_SELECTOR = CHECKBOXES(
            name                            = 'BACKTEST_SELECTOR',
            options                         = self._settings_all_backtests,
            default                         = False,
            width                           = '60%',
        )
            
        ####################################
        # instantiate BOOK_SELECTOR
        ####################################
        # needs
        # - BACKTEST_SELECTOR[books]
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - BACKTEST_SELECTOR[books]                                                                        grab correct books
        ####################################
        self._BOOK_SELECTOR = CHECKBOXES(
            name                            = 'BOOK_SELECTOR',
            options                         = self._settings_all_books, # TODO `BACKTEST SELECTOR` needs to have `all_books`
            default                         = True,
            width                           = '20%',
        )

        ####################################
        # instantiate BACKTEST_VIEWER
        ####################################
        # needs
        # - SETTINGS[view_type]
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - SETTINGS[view_type]                                                                                                    update plot device
        # - SETTINGS[plot_method]                                                                                                  update plot device
        # - SETTINGS[return_type]                                                                                                  update plot device
        ####################################

        ####################################
        # instantiate DATA_CONTAINER
        ####################################
        # needs
        # - view_type
        # - selected backtests
        # - books
        # - date / time filter params
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - creates df for plotter (TAB_COLLECTION)
        ####################################
        self._DATA_CONTAINER = DATA_CONTAINER(
            
            # options
            selected_backtests              = self._selected_backtests,
            selected_books                  = self._selected_books,

            # reference
            plot_method                     = self._settings_plot_method,
            return_type                     = self._settings_return_type,
            view_type                       = self._settings_view_type,
            date_from                       = self._settings_date_from,
            date_to                         = self._settings_date_to,
            time_from                       = self._settings_date_from,
            time_to                         = self._settings_date_to,
        )
        
        ####################################
        # build GUI
        ####################################
        self.reference                      = {}
        self.widget                         = self._make_widget(self._settings_view_type)

        ####################################
        # class linking
        ####################################
        self._SETTINGS.attach(self)                           # self._BACKTEST_VIEWER observes self._SETTINGS and coordinates messages received from it
        self._BACKTEST_SELECTOR.attach(self)                  # self._BACKTEST_VIEWER observes self._BACKTEST_SELECTOR and updates self._DATA_CONTAINER off it
        self._BOOK_SELECTOR.attach(self)                      # self._BACKTEST_VIEWER observes self._BOOK_SELECTOR and updates self._DATA_CONTAINER off it

    ####################################
    # observer pattern
    ####################################
    # observer
    def react(self,
        subject_name : str,
        subject_info : object
        ) -> None :
        logging.info(f'OBSERVER PATTERN : {self.name} REACTS subject_name {subject_name}')
        logging.info(f'OBSERVER PATTERN : {self.name} REACTS subject_info {subject_info.keys()}')

        # default to no need to rebuild df
        build_df       = False
        recompute_rtns = False
        redraw_plots   = False
        
        ###################
        # SETTINGS notified
        ###################
        if subject_name == 'SETTINGS':
            logging.info(f'{self.name} : react {subject_info}')

            #=======
            # for self._BACKTEST_SELECTOR | self._BOOK_SELECTOR
            #=======
            if ('all_backtests' in subject_info):
                logging.debug(f'{self.name} : all_backtests')
                
                build_df                                             |= True  # available backtests changed, grab data
                recompute_rtns                                       |= True  # available backtests changed, recompute returns
                redraw_plots                                         |= True  # available backtests changed, redraw plots
                
                # update self._BACKTEST_SELECTOR options
                self._BACKTEST_SELECTOR.update_options(subject_info['all_backtests'])
                self._DATA_CONTAINER.reference['selected_backtests'] = self._BACKTEST_SELECTOR.applied_settings # tell self._DATA_CONTAINER backtests have changed
                
                # update self._BOOK_SELECTOR options
                # all_books=[] # identify all books TODO
                self._BOOK_SELECTOR.update_options(subject_info['all_books']) # TODO, need to compute this from family of backtests
                self._DATA_CONTAINER.reference['selected_books']     = self._BOOK_SELECTOR.applied_settings     # tell self._DATA_CONTAINER books have changed
                
            if ('app_name' in subject_info):
                logging.debug(f'{self.name} : app_name')
                
                self._header_widget.value=self._header(subject_info['app_name'])
                
                build_df                                            |= False # renaming app, data unchanged
                recompute_rtns                                      |= False # renaming app, data unchanged
                redraw_plots                                        |= False # renaming app, data unchanged
                
            #=======
            # for self._DATA_CONTAINER
            #=======
            if ('view_type' in subject_info):
                logging.debug(f'{self.name} : view_type')

                self._DATA_CONTAINER.reference['view_type']          = subject_info['view_type']
                self._DATA_CONTAINER.data_dico.clear()
                
                build_df                                            |= True  # view_type changed, must rebuild df as data different + update plotting device
                recompute_rtns                                      |= False # view_type changed, return compute unchanged
                redraw_plots                                        |= True  # view_type changed, plotting devices change for sure
                
                # plotting tool needs to change how it looks
                self._TAB_COLLECTION = self._make_tab_collection(subject_info['view_type']) # make new tab collection
                self._tab_widgets = self._TAB_COLLECTION.widget.children                    # update frontend

            # min-date changed
            if ('date_from' in subject_info):
                logging.debug(f'{self.name} : date_from')

                self._DATA_CONTAINER.reference['date_from']          = subject_info['date_from']
                
                build_df                                            |= True  # period changed, data changes
                recompute_rtns                                      |= True  # period changed, returns change
                redraw_plots                                        |= True  # period changed, plots change
            
            # max-date changed
            if ('date_to' in subject_info):
                logging.debug(f'{self.name} : date_to')
                
                self._DATA_CONTAINER.reference['date_to']            = subject_info['date_to']
                build_df                                            |= True  # period changed, data changes
                recompute_rtns                                      |= True  # period changed, returns change
                redraw_plots                                        |= True  # period changed, plots change
            
            # min-time changed
            if ('time_from' in subject_info) & (self._DATA_CONTAINER.reference['view_type'] == 'Intraday'):
                logging.debug(f'{self.name} : time_from')
                
                self._DATA_CONTAINER.reference['time_from']          = subject_info['time_from']
                build_df                                            |= True  # period changed, data changes
                recompute_rtns                                      |= True  # period changed, returns change
                redraw_plots                                        |= True  # period changed, plots change
            
            # max-time changed
            if ('time_to' in subject_info) & (self._DATA_CONTAINER.reference['view_type'] == 'Intraday'):
                logging.debug(f'{self.name} : time_to')

                self._DATA_CONTAINER.reference['time_to']            = subject_info['time_to']
                build_df                                            |= True  # period changed, data changes
                recompute_rtns                                      |= True  # period changed, returns change
                redraw_plots                                        |= True  # period changed, plots change
            
            # asked to repopulate data
            if ('reload_data' in subject_info):
                logging.debug(f'{self.name} : reload_data')

                self._DATA_CONTAINER.data_dico.clear()
                build_df                                            |= True  # data refreshed, recompute all
                recompute_rtns                                      |= True  # data refreshed, recompute all
                redraw_plots                                        |= True  # data refreshed, recompute all

            #=======
            # for self._TAB_COLLECTION
            #=======
            if ('plot_method' in subject_info):
                self._DATA_CONTAINER.reference['plot_method']        = subject_info['plot_method']
                build_df                                            |= False # plot_method changed, no impact to data
                recompute_rtns                                      |= False # plot_method changed, no impact to returns
                redraw_plots                                        |= True  # plot_method changed, redo plots

            if ('return_type' in subject_info):
                self._DATA_CONTAINER.reference['return_type']        = subject_info['return_type']
                build_df                                            |= False # return_type changed, no impact to data
                recompute_rtns                                      |= True  # return_type changed, redo returns
                redraw_plots                                        |= False # return_type changed, no impact to plots

        ###################
        # BACKTEST_SELECTOR notified
        ###################
        if subject_name == 'BACKTEST_SELECTOR':
            # backtest selection changed
            logging.info(f'{self.name} : backtests changed')

            self._DATA_CONTAINER.reference['selected_backtests']     = subject_info
            build_df                                                |= True  # data refreshed, recompute all
            recompute_rtns                                          |= True  # data refreshed, recompute all
            redraw_plots                                            |= True  # data refreshed, recompute all

        ###################
        # BOOK_SELECTOR notified
        ###################
        if subject_name == 'BOOK_SELECTOR':
            # book selection changed
            logging.info(f'{self.name} : books changed')

            self._DATA_CONTAINER.reference['selected_books']         = subject_info
            build_df                                                |= True  # data refreshed, recompute all
            recompute_rtns                                          |= True  # data refreshed, recompute all
            redraw_plots                                            |= True  # data refreshed, recompute all

        ###################
        # rebuild df as needed
        ###################
        if build_df:
            logging.info(f'{self.name} : build_df')

            self._DATA_CONTAINER.build_df()

        ###################
        # redraw plots as needed
        ###################
        if redraw_plots:
            logging.info(f'{self.name} : redraw_plots')
            
            self._TAB_COLLECTION.update_collection(
                self._DATA_CONTAINER.df,
                self._DATA_CONTAINER.reference['plot_method'],
            )

        ###################
        # recompute returns as needed
        ###################
        if recompute_rtns:
            logging.info(f'{self.name} : recompute_rtns')

            # TODO
        
    def _header(self,txt,fontsize=50):
        return "<h1 style='font-size:"+str(fontsize)+"px'>"+txt+"</h1>"

    ####################################
    # build GUI
    ####################################
    def _make_widget(self,view_type : str) -> VBox :
        
        # header
        header = HTML(self._header(self._SETTINGS.reference['app_name']))

        # controls
        controls = HBox(
            [
                self._BACKTEST_SELECTOR.widget,
                self._BOOK_SELECTOR.widget,
                self._SETTINGS.widget,
            ]
        )
        
        # make viewing tabs
        self._TAB_COLLECTION = self._make_tab_collection(view_type)

        # build screen
        return VBox(
            [
                header,
                controls,
                self._TAB_COLLECTION.widget
            ]
        )
    
    def _make_tab_collection(self,view_type : str) -> TAB_COLLECTION :
        self.reference['applied_view_type'] = view_type                                   # update reference
        func                                = BACKTEST_VIEWER._view_type_dico[view_type]  # get class function
        logging.info(f'{self.name} : _make_tab_collection {view_type} {func}')            # debugging
        return eval(func)                                                                 # make and return it

    ####################################
    # properties
    ####################################
    # from self._SETTINGS
    @property
    def _settings_backtest_path(self) -> str :
        return self._SETTINGS.reference['backtest_path']

    @property
    def _settings_plot_method(self) -> str :
        return self._SETTINGS.reference['plot_method']

    @property
    def _settings_return_type(self) -> str :
        return self._SETTINGS.reference['return_type']

    @property
    def _settings_view_type(self) -> str :
        return self._SETTINGS.reference['view_type']

    @property
    def _settings_date_from(self) -> object :
        return self._SETTINGS.reference['date_from']

    @property
    def _settings_date_to(self) -> object :
        return self._SETTINGS.reference['date_to']

    @property
    def _settings_all_backtests(self) -> ID_PATHS :
        return self._SETTINGS.reference['all_backtests']

    @property
    def _settings_all_books(self) -> ID_PATHS :
        return self._SETTINGS.reference['all_books']

    # from self._BACKTEST_SELECTOR
    @property
    def _selected_backtests(self) -> OPTIONID_VALUES:
        return self._BACKTEST_SELECTOR.applied_settings
        
    # from self._BOOK_SELECTOR
    @property
    def _selected_books(self) -> OPTIONID_VALUES:
        return self._BOOK_SELECTOR.applied_settings

    # from self.widget
    @property
    def _header_widget(self) -> HTML :
        return self.widget.children[0]
    
    @property
    def _control_widgets(self) -> BUTTONS :
        return self.widget.children[1].children
    
    @property
    def _tab_widgets(self) -> TAB_COLLECTION :
        return self.widget.children[2].children

    @_tab_widgets.setter
    def _tab_widgets(self,new_tab_collection : TAB_COLLECTION) -> None:
        self.widget.children[2].children = new_tab_collection

In [ ]:
# __file__ exists if notebook called with %run but doesnt it called manually
# e.g. I only wish to run the example when calling this notebook directly
try:
    __file__
except NameError:
    # example
    a=BACKTEST_VIEWER()
    display(dir(a))
    display(a.widget)

In [ ]:
'''
a._DATA_CONTAINER.data_dico

# a._DATA_CONTAINER.df.to_pickle('df_extraday.pkl') # save it
x=a._DATA_CONTAINER.df # show it
x

x=x.set_index(['date'],append=True).reorder_levels([0,2,1])
x

import pickle
with open("df_in.pkl","wb") as f:
    pickle.dump(x,f)

a._DATA_CONTAINER.df.reset_index().set_index(['date'],append=True)

a._DATA_CONTAINER.df.reset_index().set_index(['backtest','date','timestamp'])

x=a._DATA_CONTAINER.df.reset_index().set_index(['backtest','date','timestamp'])
import pickle
with open("df_intraday10.pkl","wb") as f:
    pickle.dump(x,f)
    
#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)


# a._DATA_CONTAINER.df.to_pickle('df_intraday.pkl') # save it
a._DATA_CONTAINER.df
'''
None

In [ ]:
# save
'''
# df
a._DATA_CONTAINER.df.to_pickle('df_extraday.pkl')
a._DATA_CONTAINER.df.to_pickle('df_intraday.pkl')

# datadico
a._DATA_CONTAINER.data_dico.to_pickle('datadico_extraday.pkl')
a._DATA_CONTAINER.data_dico.to_pickle('datadico_intraday.pkl')

'''
None

In [ ]:
# read
'''
df_intraday = pd.read_pickle('df_intraday.pkl')
'''
None

ideas
- days of the week plot
- compound plots
- single plot needs to be easy to configure